In [4]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import math
from sklearn import svm

# Read Data

In [10]:
data = pd.read_csv('fake.csv')

split = int(math.ceil(len(data)*0.8))
train_data = data[:split]
test_data = data[split+1:]
data[:2]

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.0,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.0,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias


# Preprocessed Data

In [6]:
share_count_train = train_data['shares'].values.tolist();
train_sc = np.array([0 if math.isnan(i) else i for i in share_count_train])
train_rc = np.array([0 if math.isnan(i) else i for i in train_data['likes'].values.tolist()])
train_cc = np.array([0 if math.isnan(i) else i for i in train_data['comments'].values.tolist()])
rating = train_data['type'].unique()
print(rating)
myclass = []
name2id = {}
for val in train_data['type'].values.tolist():
    for i,val_ref in enumerate(rating):
        if val == val_ref:
            myclass.append(i)
            name2id[i] = val_ref

normalized_sc_train = (train_sc-min(train_sc))/(max(train_sc)-min(train_sc))
normalized_rc_train = (train_rc-min(train_rc))/(max(train_rc)-min(train_rc))
normalized_cc_train = (train_cc-min(train_cc))/(max(train_cc)-min(train_cc))
features_train =  np.column_stack((normalized_sc_train,normalized_rc_train,normalized_cc_train))

#plt.scatter(myclass,normalized_sc)
#plt.show()

['bias' 'conspiracy' 'fake' 'bs' 'satire' 'hate' 'junksci' 'state']


# Model fit

In [7]:
clf = svm.SVC()
clf.fit(features_train, myclass)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# Test Model

In [8]:
share_count_test = test_data['shares'].values.tolist();
test_sc = np.array([0 if math.isnan(i) else i for i in share_count_test])
test_rc = np.array([0 if math.isnan(i) else i for i in test_data['likes'].values.tolist()])
test_cc = np.array([0 if math.isnan(i) else i for i in test_data['comments'].values.tolist()])

normalized_sc_test = (test_sc-min(test_sc))/(max(test_sc)-min(test_sc))
normalized_rc_test = (test_rc-min(test_rc))/(max(test_rc)-min(test_rc))
normalized_cc_test = (test_cc-min(test_cc))/(max(test_cc)-min(test_cc))
features_test =  np.column_stack((normalized_sc_test,normalized_rc_test,normalized_cc_test))

myclass_test = []
for val in test_data['type'].values.tolist():
    for i,val_ref in enumerate(rating):
        if val == val_ref:
            myclass_test.append(i)
#print(myclass_test)

# Predict

In [9]:
output = clf.predict(features_test)
#for i in output:
 #   print(name2id[i])
    
clf.score(features_test, myclass_test)

0.9230177059276367